# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [3]:
customer_prod = data.groupby(['CustomerID','ProductName']).agg({'Quantity':sum}).reset_index()
customer_prod.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [4]:
customer_prod_pivot = customer_prod.pivot_table(index='ProductName',columns='CustomerID',values='Quantity',aggfunc='sum').fillna(0)
customer_prod_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [5]:
customer_prod_pivot_t = customer_prod_pivot.T
customer_prod_pivot_t.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [6]:
#cust_prod_pivot_norm = customer_prod_pivot.div(customer_prod_pivot.sum(axis=1),axis=0)
cust_prod_pivot_t_norm = customer_prod_pivot_t.div(customer_prod_pivot_t.sum(axis=1),axis=0)

In [7]:
cust_prod_pivot_t_norm.index

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [8]:
dist = squareform(pdist(cust_prod_pivot_t_norm,'euclidean'))
afinidad = 1/(1+dist)
afinidad_df = pd.DataFrame(afinidad, 
                           index=cust_prod_pivot_t_norm.index,  #son los customer id's
                           columns=cust_prod_pivot_t_norm.index)

afinidad_df.head() #similarity matrix 1000x1000

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [44]:
#This function receives a customer ID and returns a list of five recommendations based on similar customers.

def get_recommendation(ID):
    
    #List containing the 5 customer IDs of the customers whose purchase behavior is most similar
    
    top_most_similar = list(afinidad_df[ID].sort_values(ascending=False)[1:].head().index)
    
    top_most_similar_df = customer_prod[customer_prod.CustomerID.isin(top_most_similar)]
    
    grouped = similar_customer_prod.groupby('ProductName').agg({'Quantity':'sum'}) 
    
    ranked_prod = grouped.sort_values('Quantity',ascending=False).reset_index()
    
    # Merging the list with the custormer-products df
    
    merged = pd.merge(ranked_prod, pd.DataFrame(customer_prod_pivot_t.T[ID]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    
    #return only top 5 prod
    return merged[merged['Purchased']==0].head()['ProductName'].tolist()

In [36]:
# The recommendation dictionary

get_recommendation = {x: get_recommendation(x) for x in afinidad_df.index}

In [37]:
cust_recommendation = pd.DataFrame(get_recommendation, index=['Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']).T
cust_recommendation.index.name = 'CustomerID'
cust_recommendation.head()

,Rec1,Rec2,Rec3,Rec4,Rec5
CustomerID,,,,,
33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
200,Wine - Redchard Merritt,Bread - Calabrese Baguette,Wine - Crozes Hermitage E.,"Thyme - Lemon, Fresh",Wine - Sogrape Mateus Rose
264,Cassis,Bread - Calabrese Baguette,Wine - Crozes Hermitage E.,"Thyme - Lemon, Fresh",Veal - Eye Of Round
356,Wine - Redchard Merritt,Cassis,Bread - Calabrese Baguette,Wine - Crozes Hermitage E.,Wine - Sogrape Mateus Rose
412,Wine - Redchard Merritt,Cassis,Bread - Calabrese Baguette,Wine - Crozes Hermitage E.,"Thyme - Lemon, Fresh"


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [12]:
random_id = data.CustomerID.sample(1).iloc[0]

similar_customer = list(afinidad_df[33].sort_values(ascending=False)[1:].head().index)
similar_customer

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [13]:
#select just the purchases where the customer ID is in our list of similar customers
similar_customer_prod = customer_prod[customer_prod['CustomerID'].isin(similar_customer)] 

#aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers
grouped = similar_customer_prod.groupby('ProductName').agg({'Quantity':'sum'})

#we rank them by sorting in descending order by the total quantity.
ranked_prod = grouped.sort_values('Quantity',ascending=False).reset_index() 

ranked_prod.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [14]:
merged = pd.merge(ranked_prod, pd.DataFrame(customer_prod_pivot_t.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recommendations = merged[merged['Purchased']==0].head()
recommendations

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [18]:
prod_similarity = pd.DataFrame(1/(1 + squareform(pdist(customer_prod_pivot_t.T))), 
                               columns=customer_prod_pivot_t.columns, 
                                index=customer_prod_pivot_t.columns)
prod_similarity.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.003559,0.003927,0.004175,0.004102,0.003616,0.003767,0.003876,0.004024,0.004023,...,0.003794,0.003749,0.003846,0.004037,0.003839,0.004026,0.003908,0.003896,0.003860,0.003578
"Appetizer - Mini Egg Roll, Shrimp",0.003559,1.000000,0.003670,0.003697,0.003692,0.003413,0.003479,0.003900,0.003789,0.003486,...,0.003639,0.003481,0.003624,0.003751,0.003639,0.003508,0.003691,0.003749,0.003564,0.003475
Appetizer - Mushroom Tart,0.003927,0.003670,1.000000,0.004234,0.003849,0.003789,0.003819,0.004120,0.004110,0.003686,...,0.003920,0.003683,0.003812,0.004418,0.004155,0.004111,0.004095,0.004060,0.003937,0.003698
Appetizer - Sausage Rolls,0.004175,0.003697,0.004234,1.000000,0.004078,0.003974,0.003808,0.004169,0.003998,0.003837,...,0.003922,0.003853,0.003962,0.004551,0.004089,0.003872,0.004082,0.004020,0.003878,0.003718
Apricots - Dried,0.004102,0.003692,0.003849,0.004078,1.000000,0.003755,0.003722,0.004073,0.003895,0.003778,...,0.003760,0.003669,0.003869,0.004233,0.003836,0.003718,0.004018,0.003955,0.003811,0.003767


### Step 2: Get the products purchased for a specific customer of your choice.

In [19]:
random_id = data.CustomerID.sample(1).iloc[0]

random_id_pp = customer_prod[customer_prod.CustomerID == random_id]

random_id_pp.head()

,CustomerID,ProductName,Quantity
55207,85524,Apricots - Halves,22
55208,85524,Bar - Granola Trail Mix Fruit Nut,22
55209,85524,Bay Leaf,22
55210,85524,Beef - Ground Medium,22
55211,85524,Beef - Montreal Smoked Brisket,22


### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [20]:
top5_prod = random_id_pp.ProductName.map(lambda x: prod_similarity[x].sort_values(ascending=False)[1:6].index.tolist())
top5_prod = [p for lis in top5_prod for p in lis]   # Flattening the list

products_not_purchased = [p for p in top5_prod if not p in random_id_pp.ProductName.unique()]

### Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [21]:
top5_rec_prod = pd.Series(list(products_not_purchased)).value_counts().head().index.tolist()
top5_rec_prod

['Rabbit - Whole',
 'Water - Green Tea Refresher',
 'Scallops 60/80 Iqf',
 'Pork - Loin, Center Cut',
 'Juice - Cranberry, 341 Ml']

### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [39]:
# The get_product_rec receives a Customer ID and returns a list of five products recomended 
#based on products bought by similar customers.

def get_product_rec(ID):
    
    # Filtering the Customer-Products dataframe by ID
    random_id_pp = customer_prod[customer_prod.CustomerID == ID]
    
    # Getting a list of recommended products not purchased yet
    top5_prod = random_id_pp.ProductName.map(lambda x: prod_similarity[x].sort_values(ascending=False)[1:6].index.tolist())
    top5_prod = [p for lis in top5_prod for p in lis] 
    products_not_purchased = [p for p in top5_prod if not p in random_id_pp.ProductName.unique()]
    
    # Returning top 5 rec prod
    return pd.Series(list(products_not_purchased)).value_counts().head().index.tolist()

In [43]:
# The recommendations dict is created
get_product_rec = {x: get_product_rec(x) for x in afinidad_df.index}

### Step 6: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [45]:
product_rec = pd.DataFrame(get_product_rec, index=['Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']).T
product_rec.index.name = 'CustomerID'
product_rec.head()

,Rec1,Rec2,Rec3,Rec4,Rec5
CustomerID,,,,,
33,Rabbit - Whole,Scallops 60/80 Iqf,Wine - Wyndham Estate Bin 777,Bar - Granola Trail Mix Fruit Nut,Steam Pan - Half Size Deep
200,Rabbit - Whole,Water - Green Tea Refresher,Scallops 60/80 Iqf,Bread - Hot Dog Buns,"Pork - Loin, Center Cut"
264,Water - Green Tea Refresher,Rabbit - Whole,Scallops 60/80 Iqf,Rice - Long Grain,"Juice - Cranberry, 341 Ml"
356,Water - Green Tea Refresher,Scallops 60/80 Iqf,"Pork - Loin, Center Cut",Table Cloth 120 Round White,Bread - Hot Dog Buns
412,Water - Green Tea Refresher,Scallops 60/80 Iqf,Rabbit - Whole,Raspberries - Fresh,Bar - Granola Trail Mix Fruit Nut


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [25]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

In [26]:
# The new customer is added to the main df
new_customer_df = pd.DataFrame({'CustomerID':[696]*12, 'FirstName':['New']*12, 'LastName':['Customer']*12, 'SalesID':[None]*12, 
                             'ProductID':[None]*12, 'ProductName': list(new_customer.keys()), 'Quantity': list(new_customer.values())})

data = pd.concat([new_customer_df, data])
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,696,New,Customer,None,None,Cookies - Assorted,3
1,696,New,Customer,None,None,Flavouring - Orange,3
2,696,New,Customer,None,None,Fenngreek Seed,1
3,696,New,Customer,None,None,Wine - White Cab Sauv.on,1
4,696,New,Customer,None,None,Bandage - Flexible Neon,3


In [34]:
# Updating all the matrices with the new customer in the db.

customer_prod = data.groupby(['CustomerID','ProductName']).agg({'Quantity':sum}).reset_index()

customer_prod_pivot = customer_prod.pivot_table(index='ProductName',columns='CustomerID',values='Quantity',aggfunc='sum').fillna(0)

customer_prod_pivot_t = customer_prod_pivot.T

cust_prod_pivot_t_norm = customer_prod_pivot_t.div(customer_prod_pivot_t.sum(axis=1),axis=0)

dist = squareform(pdist(cust_prod_pivot_t_norm,'euclidean'))
afinidad = 1/(1+dist)
afinidad_df = pd.DataFrame(afinidad, 
                           index=cust_prod_pivot_t_norm.index,  #son los customer id's
                           columns=cust_prod_pivot_t_norm.index)

prod_similarity = pd.DataFrame(1/(1 + squareform(pdist(customer_prod_pivot_t.T))), 
                               columns=customer_prod_pivot_t.columns, 
                                index=customer_prod_pivot_t.columns)

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

In [46]:
# Calling previous function to get the recomendations for the new customer by similar customers.
get_recommendation(696)


['Wine - Redchard Merritt',
 'Cassis',
 'Bread - Calabrese Baguette',
 'Wine - Crozes Hermitage E.',
 'Thyme - Lemon, Fresh']

### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [41]:
# Calling previous function to get the recomendations for the new customer by similar products.

get_product_rec(696)

['Scallops 60/80 Iqf',
 'Water - Green Tea Refresher',
 'Pork - Loin, Center Cut',
 'Raspberries - Fresh',
 'Rabbit - Whole']